In [45]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [46]:
import csv

Xy = []
with open("winequality-red.csv") as fp:
    for row in csv.reader(fp, delimiter=";"):
        Xy.append(row)

In [47]:
Xy = np.array(Xy[1:], dtype=np.float64)

In [48]:
train_X = Xy[:-1000, :-1]
train_y = Xy[:-1000,-1]
test_X = Xy[-1000:,:-1]
test_y = Xy[-1000:,-1]

In [49]:
class Lasso:
    def __init__(self, lamda_, tol=0.001, max_iter=1000):
        self.lamda_ = lamda_
        self.tol = tol
        self.max_iter = max_iter
        self.w_ = None
        
    def soft_thresholding(x, y):
        return np.sign(x) * max(abs(x) - y, 0)
                
    def _update(self, row, col, X, t):
        self.w_[0] = (t - np.dot(X, self.w_[1:])).sum() / row
        w0vec = np.ones(row) * self.w_[0]
        for k in range(col):
            ww = self.w_[1:]
            ww[k] = 0
            q = np.dot(t - w0vec - np.dot(X, ww), X[:,k])
            r = np.dot(X[:,k], X[:,k])
            self.w_[k+1] = soft_thresholding(q / r, self.lamda_)
    
    def fit(self, X, t):
        row, col = X.shape
        self.w_ = np.zeros(col + 1)
        avgl1 = 0.
        for _ in range(self.max_iter):
            avgl1_prev = avgl1
            self._update(row, col, X, t)
            avgl1 = np.abs(self.w_).sum() / self.w_.shape[0]
            if abs(avgl1 - avgl1_prev) <= self.tol:
                break
    
    def predict(self, X):
        if X.ndim == 1:
            X = X.reshape(X.shape[0], 1)
        
        Xtil = np.c_[np.ones(X.shape[0]), X]
        return np.dot(Xtil, self.w_)

In [51]:
for lambda_ in [1.,0.1, 0.01]:
    model = Lasso(lambda_)
    model.fit(train_X, train_y)
    y = model.predict(test_X)
    print(f"--- lambda = {lambda_} ---")
    print("coefficients:")
    print(model.w_)
    mse = ((y - test_y)**2).mean()
    print("MSE: {:.3f}".format(mse))

--- lambda = 1.0 ---
coefficients:
[ 5.60934891  0.         -0.          0.          0.         -0.
 -0.         -0.          0.         -0.          0.          0.        ]
MSE: 0.667
--- lambda = 0.1 ---
coefficients:
[ 5.63436792  0.         -0.10331221  0.55205203  0.         -1.57557842
 -0.         -0.         -0.         -0.          0.          0.        ]
MSE: 0.617
--- lambda = 0.01 ---
coefficients:
[ 5.50360587  0.         -0.85879623  0.38621155  0.         -3.05948342
 -0.         -0.          0.10897095  0.          0.86365158  0.        ]
MSE: 0.528
